In [0]:
# HOMEWORK PART 2 - LOAD GOLD TABLE AND CHECK COLUMNS
print("=== HOMEWORK PART 2 - LOADING GOLD TABLE ===")

from pyspark.sql.functions import *

# Set storage authentication
spark.conf.set(
    "fs.azure.account.key.goodreadsreviews60301511.dfs.core.windows.net",
    "QZzPMlZcQvM/LeucwJ67H1zRkEhbWCH9+uxdJaTWALJU/QN8ArtpEhMHmVb7vT2DaXAgMY52PkPH+AStsA7+fw=="
)

# Load the curated_reviews_gold from ADLS gold layer - using Delta format
curated_reviews_gold = spark.read.format("delta").load(
    "abfss://lakehouse@goodreadsreviews60301511.dfs.core.windows.net/gold/curated_reviews/"
)

print(f"✓ Loaded gold table: {curated_reviews_gold.count()} records")
print("Available columns:")
print(curated_reviews_gold.columns)
print("\nSchema:")
curated_reviews_gold.printSchema()
print("\nSample data:")
curated_reviews_gold.show(5, truncate=False)

=== HOMEWORK PART 2 - LOADING GOLD TABLE ===
✓ Loaded gold table: 19688803 records
Available columns:
['review_id', 'book_id', 'title', 'author_id', 'name', 'user_id', 'rating', 'review_text']

Schema:
root
 |-- review_id: string (nullable = true)
 |-- book_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- author_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- rating: integer (nullable = true)
 |-- review_text: string (nullable = true)


Sample data:
+--------------------------------+--------+---------------------------------------------+---------+---------------+--------------------------------+------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
# STEP 1: DATA TYPE CONVERSION
print("=== STEP 1: DATA TYPE CONVERSION ===")

# Start with the gold table
df = curated_reviews_gold

# Convert all ID columns to string type
df = df.withColumn("review_id", col("review_id").cast("string"))
df = df.withColumn("book_id", col("book_id").cast("string"))
df = df.withColumn("author_id", col("author_id").cast("string"))
df = df.withColumn("user_id", col("user_id").cast("string"))

# Convert numeric columns (only convert what exists)
df = df.withColumn("rating", col("rating").cast("integer"))

# Add missing columns with default values if they don't exist
if 'n_votes' not in df.columns:
    df = df.withColumn("n_votes", lit(0))
else:
    df = df.withColumn("n_votes", col("n_votes").cast("integer"))

# Convert date column if it exists
if 'date_added' in df.columns:
    df = df.withColumn("date_added", to_date(col("date_added"), "yyyy-MM-dd"))

print("✓ Data types converted")
df.printSchema()

=== STEP 1: DATA TYPE CONVERSION ===
✓ Data types converted
root
 |-- review_id: string (nullable = true)
 |-- book_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- author_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- rating: integer (nullable = true)
 |-- review_text: string (nullable = true)
 |-- n_votes: integer (nullable = false)



In [0]:
# STEP 2: HANDLE MISSING VALUES
print("=== STEP 2: HANDLE MISSING VALUES ===")

initial_count = df.count()
print(f"Starting with {initial_count} records")

# Remove rows with null critical fields (only check columns that exist)
filter_condition = (
    col("review_id").isNotNull() &
    col("book_id").isNotNull() &
    col("user_id").isNotNull() &
    col("rating").isNotNull() &
    col("review_text").isNotNull()
)

df = df.filter(filter_condition)

# Remove invalid ratings (only keep 1-5)
df = df.filter((col("rating") >= 1) & (col("rating") <= 5))

# Handle missing values for optional columns
if 'language' in df.columns:
    df = df.withColumn("language", when(col("language").isNull(), "Unknown").otherwise(col("language")))

# Remove future dates if date_added exists
if 'date_added' in df.columns:
    df = df.filter(col("date_added") <= current_date())

print(f"✓ Removed {initial_count - df.count()} invalid records")
print(f"Remaining records: {df.count()}")

=== STEP 2: HANDLE MISSING VALUES ===
Starting with 19688803 records
✓ Removed 0 invalid records
Remaining records: 19688803


In [0]:
# STEP 3: TEXT STANDARDIZATION
print("=== STEP 3: TEXT STANDARDIZATION ===")

# Trim whitespace from all text columns
df = df.withColumn("review_text", trim(col("review_text")))
df = df.withColumn("title", trim(col("title")))
df = df.withColumn("name", trim(col("name")))

# Remove very short reviews (< 10 characters)
df = df.withColumn("review_length", length(col("review_text")))
df = df.filter(col("review_length") >= 10)

# Capitalize Each Word for title and author name
df = df.withColumn("title", initcap(col("title")))
df = df.withColumn("name", initcap(col("name")))

print("✓ Text standardization completed")
print(f"Records after text cleaning: {df.count()}")
df.select("title", "name", "rating", "review_length").show(5)

=== STEP 3: TEXT STANDARDIZATION ===
✓ Text standardization completed
Records after text cleaning: 19688803
+--------------------+--------------------+------+-------------+
|               title|                name|rating|review_length|
+--------------------+--------------------+------+-------------+
|The Girl On The T...|       Paula Hawkins|     4|          246|
|      Eating Animals|Jonathan Safran Foer|     4|          396|
|    The Tiger's Wife|          Tea Obreht|     5|           45|
|    Ethel And Ernest|      Raymond Briggs|     2|          135|
|Wild Card (north ...|        Karina Halle|     5|         1284|
+--------------------+--------------------+------+-------------+
only showing top 5 rows


In [0]:
# STEP 4: FEATURE ENGINEERING
print("=== STEP 4: FEATURE ENGINEERING ===")

# Add text analysis features
df = df.withColumn("word_count", size(split(col("review_text"), " ")))
df = df.withColumn("sentence_count", size(split(col("review_text"), "[.!?]+")) - 1)

# Add sentiment classification
df = df.withColumn("sentiment",
    when(col("rating") >= 4, "Positive")
    .when(col("rating") <= 2, "Negative")
    .otherwise("Neutral")
)

# Add spoiler detection
df = df.withColumn("contains_spoilers",
    lower(col("review_text")).rlike("spoiler|ruined|ending|twist|surprise")
)

# Add review quality indicators (use n_votes if available)
df = df.withColumn("is_detailed_review", col("word_count") > 50)
df = df.withColumn("is_helpful_review", col("n_votes") > 5)

print("✓ Feature engineering completed")
available_cols = ["review_id", "rating", "sentiment", "word_count", "contains_spoilers"]
if 'is_helpful_review' in df.columns:
    available_cols.append("is_helpful_review")
    
df.select(available_cols).show(5)

=== STEP 4: FEATURE ENGINEERING ===
✓ Feature engineering completed
+--------------------+------+---------+----------+-----------------+-----------------+
|           review_id|rating|sentiment|word_count|contains_spoilers|is_helpful_review|
+--------------------+------+---------+----------+-----------------+-----------------+
|c65a9df89ee29c319...|     4| Positive|        47|            false|            false|
|4754b67acf2199944...|     4| Positive|        63|            false|            false|
|a73c01199f1d8d6f9...|     5| Positive|         8|            false|            false|
|a0d8905fc3e9c2ae6...|     2| Negative|        20|            false|            false|
|e9cc539916767c676...|     5| Positive|       242|            false|            false|
+--------------------+------+---------+----------+-----------------+-----------------+
only showing top 5 rows


In [0]:
# STEP 5: BOOK-LEVEL AGGREGATIONS
print("=== STEP 5: BOOK-LEVEL AGGREGATIONS ===")

# Start with basic aggregations that we know work
agg_list = [
    avg("rating").alias("avg_book_rating"),
    count("review_id").alias("total_reviews"),
    countDistinct("user_id").alias("unique_reviewers"),
    sum(when(col("sentiment") == "Positive", 1).otherwise(0)).alias("positive_reviews"),
    sum(when(col("sentiment") == "Negative", 1).otherwise(0)).alias("negative_reviews"),
    sum(when(col("contains_spoilers") == True, 1).otherwise(0)).alias("spoiler_reviews"),
    sum(when(col("is_detailed_review") == True, 1).otherwise(0)).alias("detailed_reviews")
]

# Add n_votes aggregation if the column exists
if 'n_votes' in df.columns:
    agg_list.append(sum("n_votes").alias("total_votes"))
    agg_list.append(sum(when(col("is_helpful_review") == True, 1).otherwise(0)).alias("helpful_reviews"))

# Add text analysis aggregates
agg_list.extend([
    avg("review_length").alias("avg_review_length"),
    avg("word_count").alias("avg_word_count")
])

book_aggregations = df.groupBy("book_id", "title", "name").agg(*agg_list).orderBy(col("avg_book_rating").desc())

print("Book Aggregations - Top 10:")
book_aggregations.show(10, truncate=False)

=== STEP 5: BOOK-LEVEL AGGREGATIONS ===
Book Aggregations - Top 10:
+--------+------------------------------------------------------------------+------------------+---------------+-------------+----------------+----------------+----------------+---------------+----------------+-----------+---------------+------------------+--------------+
|book_id |title                                                             |name              |avg_book_rating|total_reviews|unique_reviewers|positive_reviews|negative_reviews|spoiler_reviews|detailed_reviews|total_votes|helpful_reviews|avg_review_length |avg_word_count|
+--------+------------------------------------------------------------------+------------------+---------------+-------------+----------------+----------------+----------------+---------------+----------------+-----------+---------------+------------------+--------------+
|9921694 |Immortal                                                          |Gene Doucette     |5.0            |1

In [0]:
# STEP 6: AUTHOR-LEVEL AGGREGATIONS
print("=== STEP 6: AUTHOR-LEVEL AGGREGATIONS ===")

# Start with basic author aggregations
author_agg_list = [
    avg("rating").alias("avg_author_rating"),
    count("review_id").alias("total_author_reviews"),
    countDistinct("book_id").alias("books_reviewed"),
    avg("review_length").alias("avg_review_length"),
    avg("word_count").alias("avg_word_count"),
    sum(when(col("sentiment") == "Positive", 1).otherwise(0)).alias("positive_reviews"),
    sum(when(col("sentiment") == "Negative", 1).otherwise(0)).alias("negative_reviews")
]

# Add n_votes aggregation if the column exists
if 'n_votes' in df.columns:
    author_agg_list.append(sum("n_votes").alias("total_author_votes"))

author_aggregations = df.groupBy("author_id", "name").agg(*author_agg_list).orderBy(col("total_author_reviews").desc())

print("Author Aggregations - Top 10:")
author_aggregations.show(10, truncate=False)

=== STEP 6: AUTHOR-LEVEL AGGREGATIONS ===
Author Aggregations - Top 10:
+---------+---------------+------------------+--------------------+--------------+-----------------+------------------+----------------+----------------+------------------+
|author_id|name           |avg_author_rating |total_author_reviews|books_reviewed|avg_review_length|avg_word_count    |positive_reviews|negative_reviews|total_author_votes|
+---------+---------------+------------------+--------------------+--------------+-----------------+------------------+----------------+----------------+------------------+
|1077326  |J.k. Rowling   |4.271650829877863 |88507               |1531          |529.285435050335 |96.69500717457376 |71654           |5466            |0                 |
|3389     |Stephen King   |3.9672213753920964|79381               |4270          |623.7700583262997|114.14209949484133|56831           |7088            |0                 |
|150038   |Cassandra Clare|4.177081554834671 |70284            

In [0]:
# STEP 7: SAVE RESULTS
print("=== STEP 7: SAVE RESULTS ===")

# Save cleaned and enriched dataset to ADLS using Delta format
df.write.format("delta").mode("overwrite").save("abfss://lakehouse@goodreadsreviews60301511.dfs.core.windows.net/homework2/cleaned_reviews")
book_aggregations.write.format("delta").mode("overwrite").save("abfss://lakehouse@goodreadsreviews60301511.dfs.core.windows.net/homework2/book_aggregations")
author_aggregations.write.format("delta").mode("overwrite").save("abfss://lakehouse@goodreadsreviews60301511.dfs.core.windows.net/homework2/author_aggregations")

print("✓ All datasets saved to ADLS homework2 folder (Delta format)")

# Also save to DBFS for reliability
df.write.format("delta").mode("overwrite").save("/tmp/goodreads/homework2/cleaned_reviews")
book_aggregations.write.format("delta").mode("overwrite").save("/tmp/goodreads/homework2/book_aggregations")
author_aggregations.write.format("delta").mode("overwrite").save("/tmp/goodreads/homework2/author_aggregations")

print("✓ All datasets saved to DBFS backup (Delta format)")

# Verify files were saved
print("DBFS files:")
display(dbutils.fs.ls("/tmp/goodreads/homework2/"))

=== STEP 7: SAVE RESULTS ===
✓ All datasets saved to ADLS homework2 folder (Delta format)
✓ All datasets saved to DBFS backup (Delta format)
DBFS files:


path,name,size,modificationTime
dbfs:/tmp/goodreads/homework2/author_aggregations/,author_aggregations/,0,1762694727000
dbfs:/tmp/goodreads/homework2/book_aggregations/,book_aggregations/,0,1762694696000
dbfs:/tmp/goodreads/homework2/cleaned_reviews/,cleaned_reviews/,0,1762694627000


In [0]:
# STEP 8: DATA QUALITY REPORT
print("=== STEP 8: DATA QUALITY REPORT ===")

print("FINAL DATASET METRICS:")
print(f"Total Reviews: {df.count()}")
print(f"Unique Books: {df.select('book_id').distinct().count()}")
print(f"Unique Authors: {df.select('author_id').distinct().count()}")
print(f"Unique Users: {df.select('user_id').distinct().count()}")
print(f"Average Rating: {df.select(avg('rating')).first()[0]:.2f}")
print(f"Average Review Length: {df.select(avg('review_length')).first()[0]:.0f} characters")
print(f"Average Word Count: {df.select(avg('word_count')).first()[0]:.0f} words")

if 'n_votes' in df.columns:
    print(f"Total Votes: {df.select(sum('n_votes')).first()[0]}")

# Sentiment distribution
print("\nSENTIMENT DISTRIBUTION:")
df.groupBy("sentiment").count().orderBy("count", ascending=False).show()

# Spoiler analysis
print("SPOILER ANALYSIS:")
df.groupBy("contains_spoilers").count().show()

# Data validation checks
print("\nDATA VALIDATION CHECKS:")
print(f"✓ No null review_ids: {df.filter(col('review_id').isNull()).count() == 0}")
print(f"✓ No null book_ids: {df.filter(col('book_id').isNull()).count() == 0}")
print(f"✓ Ratings in valid range (1-5): {df.filter((col('rating') < 1) | (col('rating') > 5)).count() == 0}")
print(f"✓ All reviews have sufficient length: {df.filter(col('review_length') < 10).count() == 0}")

if 'n_votes' in df.columns:
    print(f"✓ No negative vote counts: {df.filter(col('n_votes') < 0).count() == 0}")

if 'date_added' in df.columns:
    print(f"✓ No future dates: {df.filter(col('date_added') > current_date()).count() == 0}")

=== STEP 8: DATA QUALITY REPORT ===
FINAL DATASET METRICS:
Total Reviews: 19688803
Unique Books: 2010166
Unique Authors: 655625
Unique Users: 453368
Average Rating: 3.90
Average Review Length: 726 characters
Average Word Count: 131 words
Total Votes: 0

SENTIMENT DISTRIBUTION:
+---------+--------+
|sentiment|   count|
+---------+--------+
| Positive|13560113|
|  Neutral| 4117618|
| Negative| 2011072|
+---------+--------+

SPOILER ANALYSIS:
+-----------------+--------+
|contains_spoilers|   count|
+-----------------+--------+
|             true| 3094642|
|            false|16594161|
+-----------------+--------+


DATA VALIDATION CHECKS:
✓ No null review_ids: True
✓ No null book_ids: True
✓ Ratings in valid range (1-5): True
✓ All reviews have sufficient length: True
✓ No negative vote counts: True


In [0]:
# STEP 9: FINAL SUMMARY
print("=== STEP 9: FINAL SUMMARY ===")

print("🎉 HOMEWORK PART 2 COMPLETED SUCCESSFULLY! 🎉")
print("=" * 60)
print("TRANSFORMATIONS PERFORMED:")
print("✓ Data type conversion (ID columns to string, numeric validation)")
print("✓ Missing value handling (null removal, default values)")
print("✓ Text standardization (trimming, capitalization, length validation)")
print("✓ Feature engineering (sentiment, spoiler detection, quality indicators)")
print("✓ Book-level aggregations (ratings, reviews, engagement metrics)")
print("✓ Author-level aggregations (performance, volume, sentiment analysis)")
print("✓ Data quality validation (range checks, completeness verification)")
print("=" * 60)
print("RESULTS SAVED:")
print(f"✓ Cleaned reviews: {df.count()} records")
print(f"✓ Book aggregations: {book_aggregations.count()} records")
print(f"✓ Author aggregations: {author_aggregations.count()} records")
print("=" * 60)

print("\nFINAL SAMPLE - Top Rated Books:")
book_cols = ["title", "name", "avg_book_rating", "total_reviews"]
if 'total_votes' in book_aggregations.columns:
    book_cols.append("total_votes")
book_aggregations.select(book_cols).orderBy(col("avg_book_rating").desc()).show(5, truncate=False)

print("\nFINAL SAMPLE - Most Reviewed Authors:")
author_cols = ["name", "total_author_reviews", "books_reviewed", "avg_author_rating"]
if 'total_author_votes' in author_aggregations.columns:
    author_cols.append("total_author_votes")
author_aggregations.select(author_cols).orderBy(col("total_author_reviews").desc()).show(5, truncate=False)

=== STEP 9: FINAL SUMMARY ===
🎉 HOMEWORK PART 2 COMPLETED SUCCESSFULLY! 🎉
TRANSFORMATIONS PERFORMED:
✓ Data type conversion (ID columns to string, numeric validation)
✓ Missing value handling (null removal, default values)
✓ Text standardization (trimming, capitalization, length validation)
✓ Feature engineering (sentiment, spoiler detection, quality indicators)
✓ Book-level aggregations (ratings, reviews, engagement metrics)
✓ Author-level aggregations (performance, volume, sentiment analysis)
✓ Data quality validation (range checks, completeness verification)
RESULTS SAVED:
✓ Cleaned reviews: 19688803 records
✓ Book aggregations: 2856573 records
✓ Author aggregations: 655625 records

FINAL SAMPLE - Top Rated Books:
+-------------------------------------------------+----------------+---------------+-------------+-----------+
|title                                            |name            |avg_book_rating|total_reviews|total_votes|
+-------------------------------------------------+